In [ ]:
# 定义一个函数来检测连续 30 天及以上 TMAX 或 TMIN 缺失的情况
def remove_long_missing_periods(group):
    group = group.sort_values(by='DATE').reset_index(drop=True)
    group['TMAX_missing'] = group['TMAX'].isna()
    group['TMIN_missing'] = group['TMIN'].isna()

    # 计算连续缺失的天数
    group['TMAX_missing_cumsum'] = group['TMAX_missing'].astype(int).groupby(group['TMAX_missing'].diff().ne(0).cumsum()).cumsum()
    group['TMIN_missing_cumsum'] = group['TMIN_missing'].astype(int).groupby(group['TMIN_missing'].diff().ne(0).cumsum()).cumsum()

    # 过滤掉连续缺失 30 天及以上的部分
    mask = (group['TMAX_missing_cumsum'] >= 15) | (group['TMIN_missing_cumsum'] >= 15)
    return group[~mask].drop(columns=['TMAX_missing', 'TMIN_missing', 'TMAX_missing_cumsum', 'TMIN_missing_cumsum'])

# 按照 lat 和 long 进行分组并处理
df_cleaned1 = df.groupby(['lat', 'long'], group_keys=False).apply(remove_long_missing_periods)

In [ ]:
# 计算 30 天滚动均值的自定义函数
def rolling_mean_cudf(group, col):
    group = group.sort_values(by='DATE').reset_index(drop=True)

    # 计算滚动均值
    rolling_means = group[col].fillna(method='ffill').rolling(window=30, min_periods=1).mean()

    # 用滚动均值填充缺失值
    group[col] = group[col].fillna(rolling_means)

    return group

# 按照 lat, long 分组计算并填充 TMAX、TMIN
df = df.groupby(['lat', 'long'], group_keys=False).apply(lambda g: rolling_mean_cudf(g, 'TMAX'))
df = df.groupby(['lat', 'long'], group_keys=False).apply(lambda g: rolling_mean_cudf(g, 'TMIN'))

In [ ]:
# 按经纬度分组处理
def remove_non_continuous_dates(group):
    group = group.sort_values("DATE").reset_index(drop=True)
    group["date_diff"] = group["DATE"].diff().dt.days  # 计算日期差值

    # 找到不连续的日期（date_diff > 1）
    discontinuous_rows = group[group["date_diff"] > 1]

    if not discontinuous_rows.empty:
        # 取最后一个不连续的日期（不一定是最大日期）
        last_discontinuous_date = discontinuous_rows.iloc[-1]["DATE"]
        # 删除最后一个不连续日期及之前的数据
        group = group[group["DATE"] > last_discontinuous_date]

    return group.drop(columns=["date_diff"])

# 如果 df_cleaned1 是 cudf.DataFrame，则转换为 pandas
if isinstance(df, cudf.DataFrame):
    df1 = df.to_pandas()

# 应用分组逻辑
df1 = df1.groupby(["lat", "long"], group_keys=False).apply(remove_non_continuous_dates)